In [0]:
%run "/Workspace/Users/samuel.barroscatarino@educ.sasserno.fr/musicstreamapp/databricks/01_Initialize_Setting"

## 1. Dimension: Users

In [0]:
# Process users with SCD Type 2 handling for level changes
def process_users_scd2(batch_df, batch_id):
    if batch_df.isEmpty():
        return

    # Create a temporary view of the incoming batch
    batch_df.createOrReplaceGlobalTempView("users_batch")

    # Check if dim_users table exists
    tables = (spark.sql(f"SHOW TABLES IN {catalog_name}.{silver_schema}")
              .filter(F.col("tableName") == "dim_users")
              .collect())
    if len(tables) == 0:
        # First-time table creation that handles historical changes
        # This implementation captures the history of changes already in the source data
            
        # Create a temporary view of the current batch
        spark.sql("""
                  CREATE OR REPLACE TEMPORARY VIEW users_with_change_history AS
                  WITH user_level_changes AS (
                      -- Detect level changes by comparing with previous level
                      SELECT
                        userId, 
                        firstName,
                        lastName,
                        gender,
                        level,
                        registration_time,
                        timestamp,
                        LAG(level) OVER (PARTITION BY userId ORDER BY timestamp) AS prev_level,
                        -- Flag rows where level changed or it's the first appearance of the user
                        CASE 
                            WHEN LAG(level) OVER (PARTITION BY userId ORDER BY timestamp) IS NULL THEN 1
                            WHEN level != LAG(level) OVER (PARTITION BY userId ORDER BY timestamp) THEN 1
                            ELSE 0
                        END AS level_changed
                      FROM global_temp.users_batch
                  ),
                  change_groups AS (
                      -- Assign a group number to each sequence of the same level
                      SELECT 
                        *,
                        SUM(level_changed) OVER (PARTITION BY userId ORDER BY timestamp) AS change_group
                      FROM user_level_changes
                  ),
                  change_boundaries AS (
                      -- Get the first and last timestamp for each level period
                      SELECT 
                        userId,
                        firstName,
                        lastName,
                        gender,
                        level,
                        change_group,
                        MIN(registration_time) AS registration_time,
                        MIN(timestamp) AS start_date,
                        MAX(timestamp) AS end_date
                      FROM change_groups
                      GROUP BY userId, firstName, lastName, gender, level, change_group
                  ),
                  finalized_changes AS (
                      -- Create proper activation and expiration dates for each record
                      SELECT
                        userId,
                        firstName,
                        lastName,
                        gender,
                        level,
                        registration_time,
                        CAST(date_trunc('day', start_date) AS DATE) AS row_activation_date,
                        CASE
                            WHEN LEAD(start_date) OVER (PARTITION BY userId ORDER BY start_date) IS NOT NULL
                            THEN CAST(date_trunc('day', LEAD(start_date) OVER (PARTITION BY userId ORDER BY start_date)) AS DATE)
                            ELSE CAST('9999-12-31' AS DATE)
                        END AS row_expiration_date,
                        CASE
                            WHEN LEAD(start_date) OVER (PARTITION BY userId ORDER BY start_date) IS NULL THEN 1
                            ELSE 0
                        END AS current_row,
                        md5(concat_ws('|', userId, level, start_date)) AS userKey
                      FROM change_boundaries
                  )
                  SELECT * FROM finalized_changes
                  ORDER BY userId, row_activation_date
                  """)
        
        # Write the SCD2 data to the dimension table
        (spark.table("users_with_change_history").write
         .format("delta")
         .mode("overwrite")
         .saveAsTable(f"{catalog_name}.{silver_schema}.dim_users"))
        
        print("Initialized dim_users table with historical changes")
    
    else:
        # 1. Find users with level changes
        spark.sql("""
                  CREATE OR REPLACE TEMPORARY VIEW level_changes AS
                  SELECT
                    b.userId, 
                    b.firstName, 
                    b.lastName, 
                    b.gender, 
                    b.level AS new_level, 
                    b.registration_time,
                    b.timestamp,
                    d.level AS old_level,
                    d.userKey,
                    d.row_activation_date
                  FROM global_temp.users_batch b 
                  JOIN music_streaming.silver.dim_users d 
                  ON b.userId = d.userId and d.current_row = 1
                  WHERE b.level <> d.level
                  """)
        
        # 2. Expire the current records for changed users
        spark.sql("""
                  MERGE INTO music_streaming.silver.dim_users d
                  USING level_changes c
                  ON d.userKey = c.userKey and d.current_row = 1
                  WHEN MATCHED THEN
                    UPDATE SET current_row = 0, 
                               row_expiration_date = CAST(date_trunc('day', c.timestamp) AS DATE)
                  """)
        
        # 3. Insert new records for users with level changes
        spark.sql("""
                  INSERT INTO music_streaming.silver.dim_users
                  SELECT 
                    userId,
                    firstName,
                    lastName,
                    gender,
                    new_level AS level,
                    registration_time,
                    CAST(date_trunc('day', timestamp) AS DATE) AS row_activation_date,
                    CAST('9999-12-31' AS DATE) AS row_expiration_date,
                    1 AS current_row,
                    md5(concat_ws('|', userId, new_level, timestamp)) AS userKey
                  FROM level_changes
                  """)
        
        # 4. Insert completely new users (not in the dimension yet)
        spark.sql("""
                  INSERT INTO music_streaming.silver.dim_users
                  SELECT
                    b.userId,
                    b.firstName,
                    b.lastName,
                    b.gender,
                    b.level,
                    b.registration_time,
                    CAST(date_trunc('day', b.timestamp) AS DATE) AS row_activation_date,
                    CAST('9999-12-31' AS DATE) AS row_expiration_date,
                    1 AS current_row,
                    md5(concat_ws('|', b.userId, b.level, b.timestamp)) AS userKey
                  FROM global_temp.users_batch b
                  LEFT JOIN music_streaming.silver.dim_users d
                  ON b.userId = d.userId
                  WHERE d.userId IS NULL
                  """)

In [0]:
# Use foreachBatch to implement SCD Type 2 logic for the users dimension
df_users_stream = (spark.readStream.table(f"{catalog_name}.{bronze_schema}.listen_events")
                   .select("userId", "firstName", "lastName", "gender", "level", "registration_time", "timestamp")
                   .filter(F.col("userId").isNotNull())
                   .withColumn("firstName", F.coalesce(F.col("firstName"), F.lit("Unknown")))
                   .withColumn("lastName", F.coalesce(F.col("lastName"), F.lit("Unknown")))
                   .withColumn("gender", F.coalesce(F.col("gender"), F.lit("Unknown"))))

In [0]:
# Write dim_users to Silver layer with streaming, using foreachBatch to handle SCD2
dim_users = (df_users_stream.writeStream
             .foreachBatch(process_users_scd2)
             .option("checkpointLocation", f"{checkpoint_path}/{silver_schema}/dim_users/")
             .trigger(availableNow=True)
             .start())

In [0]:
%sql
SELECT * FROM music_streaming.silver.dim_users LIMIT 5;
SELECT * FROM music_streaming.silver.dim_users WHERE current_row = 1 LIMIT 5;

In [0]:
%sql
WITH user_level_changes AS (
    -- Detect level changes by comparing with previous level
    SELECT
      userId, 
      firstName,
      lastName,
      gender,
      level,
      registration_time,
      timestamp,
      LAG(level) OVER (PARTITION BY userId ORDER BY timestamp) AS prev_level,
      -- Flag rows where level changed or it's the first appearance of the user
      CASE 
          WHEN LAG(level) OVER (PARTITION BY userId ORDER BY timestamp) IS NULL THEN 1
          WHEN level != LAG(level) OVER (PARTITION BY userId ORDER BY timestamp) THEN 1
          ELSE 0
      END AS level_changed
    FROM (SELECT * FROM music_streaming.bronze.listen_events)
) SELECT * FROM user_level_changes;

/*
,change_groups AS (
    -- Assign a group number to each sequence of the same level
    SELECT 
      *,
      SUM(level_changed) OVER (PARTITION BY userId ORDER BY timestamp) AS change_group
    FROM user_level_changes
),
change_boundaries AS (
    -- Get the first and last timestamp for each level period
    SELECT 
      userId,
      firstName,
      lastName,
      gender,
      level,
      registration_time,
      change_group,
      MIN(timestamp) AS start_date,
      MAX(timestamp) AS end_date
    FROM change_groups
    GROUP BY userId, firstName, lastName, gender, level, registration_time, change_group
),
finalized_changes AS (
    -- Create proper activation and expiration dates for each record
    SELECT
      userId,
      firstName,
      lastName,
      gender,
      level,
      registration_time,
      CAST(date_trunc('day', start_date) AS DATE) AS row_activation_date,
      CASE
          WHEN LEAD(start_date) OVER (PARTITION BY userId ORDER BY start_date) IS NOT NULL
          THEN CAST(date_trunc('day', LEAD(start_date) OVER (PARTITION BY userId ORDER BY start_date)) AS DATE)
          ELSE CAST('9999-12-31' AS DATE)
      END AS row_expiration_date,
      CASE
          WHEN LEAD(start_date) OVER (PARTITION BY userId ORDER BY start_date) IS NULL THEN 1
          ELSE 0
      END AS current_row,
      md5(concat_ws('|', userId, level, start_date)) AS userKey
    FROM change_boundaries
)
SELECT * FROM finalized_changes
ORDER BY userId, row_activation_date*/

## 2. Dimension: Location

In [0]:
location_stream = (spark.readStream.table(f"{catalog_name}.{bronze_schema}.listen_events")
                .select("city", F.col("state").alias("stateCode"), "zip", "latitude", "longitude")
                .filter(F.col("city").isNotNull() &
                        F.col("stateCode").isNotNull())
                .dropDuplicates(["city", "stateCode", "zip", "latitude", "longitude"]))

In [0]:
state_raw = spark.read.csv(f"{raw_path}/state_codes.csv", header=True)

In [0]:
location_raw = (location_stream.join(state_raw, "stateCode", "left")
                .withColumn("stateCode", F.coalesce(F.col("stateCode"), F.lit("NA")))
                .withColumn("stateName", F.coalesce(F.col("stateName"), F.lit("NA")))
                .withColumn("locationKey", F.md5(F.concat_ws("|", 
                    F.col("city"), 
                    F.col("stateCode"), 
                    F.coalesce(F.col("zip"), F.lit("Unknown"))
                ))))

In [0]:
(location_raw.writeStream
    .format("delta")
    .outputMode("append")
    .option("checkpointLocation", f"{checkpoint_path}/{silver_schema}/dim_location/")
    .trigger(availableNow=True)
    .toTable(f"{catalog_name}.{silver_schema}.dim_location"))

In [0]:
%sql
SELECT * FROM music_streaming.silver.dim_location LIMIT 5;